<a href="https://colab.research.google.com/github/Storm00212/JARVIS/blob/main/JARVIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q --upgrade \
  langchain \
  langchain-community \
  langchain-text-splitters \
  langchain-huggingface \
  sentence-transformers \
  faiss-cpu \
  pypdf \
  python-docx \
  python-pptx \
  tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.1/476.1 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not cu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

BASE_DIR = "/content/drive/MyDrive/jarvis-ai"
DATA_DIR = f"{BASE_DIR}/data/raw"
VECTORSTORE_DIR = f"{BASE_DIR}/vectorstore/faiss_index"

os.makedirs(VECTORSTORE_DIR, exist_ok=True)

print("Data dir exists:", os.path.exists(DATA_DIR))


Data dir exists: True


In [ ]:
from langchain_community.document_loaders import (
    PyPDFLoader,
    Docx2txtLoader,
    UnstructuredPowerPointLoader
)
from langchain_core.documents import Document
from tqdm import tqdm

In [ ]:
def load_documents(folder_path):
    documents = []

    for file in tqdm(os.listdir(folder_path)):
        path = os.path.join(folder_path, file)

        try:
            if file.lower().endswith(".pdf"):
                loader = PyPDFLoader(path)

            elif file.lower().endswith(".docx"):
                loader = Docx2txtLoader(path)

            elif file.lower().endswith(".pptx"):
                loader = UnstructuredPowerPointLoader(path)

            else:
                continue

            documents.extend(loader.load())

        except Exception as e:
            print(f"Failed to load {file}: {e}")

    return documents


In [ ]:
docs = load_documents(DATA_DIR)
print(f"Loaded {len(docs)} raw documents/pages")


 34%|███▍      | 28/82 [04:29<20:26, 22.72s/it]

Failed to load EEE 3207 ELECTRICAL MACHINES 2 (2).pptx: unstructured package not found, please install it with `pip install unstructured`


100%|██████████| 82/82 [09:08<00:00,  6.68s/it]

Loaded 11843 raw documents/pages


In [ ]:
# chunking
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # safe for LLMs
    chunk_overlap=200       # preserves context
)

chunks = text_splitter.split_documents(docs)
print(f"Created {len(chunks)} text chunks")

Created 26187 text chunks


In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

In [ ]:
# FAISS vector store
from langchain_community.vectorstores import FAISS

# Prepare lists for texts and metadatas, ensuring proper cleaning
texts_to_embed = []
metadatas_to_embed = []

for chunk in chunks:
    if chunk.page_content is not None:
        # Explicitly convert to str, and aggressively clean problematic characters
        content = str(chunk.page_content)
        try:
            content = content.encode("ascii", "ignore").decode("utf-8").strip()
        except (UnicodeEncodeError, UnicodeDecodeError):
            # Fallback to simple strip if encoding/decoding fails for some reason
            content = content.strip()

        if content: # Only include non-empty strings after stripping
            texts_to_embed.append(content)
            metadatas_to_embed.append(chunk.metadata)

# Create the FAISS vectorstore from texts and metadatas
vectorstore = FAISS.from_texts(
    texts=texts_to_embed,
    embedding=embeddings,
    metadatas=metadatas_to_embed
)
print(f"Created FAISS vectorstore with {len(texts_to_embed)} valid chunks.")

Created FAISS vectorstore with 26187 valid chunks.


In [ ]:
# saving vector store
vectorstore.save_local(VECTORSTORE_DIR)
print("FAISS vector store saved to Drive.")


FAISS vector store saved to Drive.


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
BASE_DIR = "/content/drive/MyDrive/jarvis-ai"
VECTORSTORE_DIR = f"{BASE_DIR}/vectorstore/faiss_index"

import os
print("FAISS exists:", os.path.exists(VECTORSTORE_DIR))


FAISS exists: True


In [6]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
vectorstore = FAISS.load_local(
    VECTORSTORE_DIR,
    embeddings,
    allow_dangerous_deserialization=True
)


In [8]:
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5}
)


In [12]:
query = "Explain Gauss's Law"

docs = retriever.invoke(query)

for i, doc in enumerate(docs):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content[:500])  # first 500 chars
    print(docs[0].metadata)



--- Result 1 ---
is taken is called aGaussian surface. Equation (3.25) is a mathematical
statement of Gausss law. Gausss law may be stated in words as follows.
The total electric ux emanating from a closed surface is numerically
equal to the net positive charge inside the closed surface. Gausss law
can also be expressed in terms of electric eld intensity in free space as

s
E  ds = Q
0
(3.27)
If the charges are distributed in a volume bounded by a surface, (3.25)
can then be written as
s
D  ds =

v
v dv (3.28)
S
{'producer': 'Acrobat Distiller 5.0.5 for Macintosh', 'creator': 'Adobe Acrobat Pro 10.1.3', 'creationdate': '2013-08-24T09:07:48+04:00', 'author': 'BPMAC1', 'moddate': '2013-10-03T18:41:25+04:00', 'source': '/content/drive/MyDrive/jarvis-ai/data/raw/guru_b-s-_hiziroglu_h-r-_electromagnetic_field_bookzz-org (2).pdf', 'total_pages': 698, 'page': 99, 'page_label': '83'}

--- Result 2 ---
Electrostatics 201
Total number of lines of force over the whole surface
=
00 0
444 s
QQ Q d